In [3]:
!pip install scikeras

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


# RandomSearchCV for classification pblm

In [2]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier

In [3]:
def create_model(activation):
  model = Sequential()
  model.add(Dense(16, input_dim=X_train.shape[1], activation=activation))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
  return model

In [5]:
df = np.loadtxt('/content/drive/MyDrive/DeepLearning/ANN/pima-indians-diabetes.csv', delimiter=',')
X = df[:, 0:8]
y = df[:, 8]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
model = KerasClassifier(model=create_model, batch_size=10, epochs=10, verbose=0, activation='relu')

In [13]:
activation = ['relu', 'linear', 'sigmoid']
param = dict(activation=activation)

random = RandomizedSearchCV(estimator=model, param_distributions=param, cv=3, n_jobs=-1)

In [14]:
random_result = random.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


In [15]:
print(random_result.best_score_, random_result.best_params_)

0.6611908177905309 {'activation': 'linear'}


In [16]:
pred_y = random.predict(X_test)
y_pred = (pred_y > 0.5)

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

In [18]:
cm

array([[91, 16],
       [39,  8]])

In [19]:
acc

0.6428571428571429

# RandomSearchCV for Regression pblm

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor

In [21]:
def create_model(neurons, kernel_initializer, optimizer='adam', loss='mean_absolute_percentage_error'):
  model = Sequential()
  model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu', kernel_initializer=kernel_initializer))
  model.add(Dense(1, activation='relu', kernel_initializer=kernel_initializer))
  model.compile(loss=loss, optimizer=optimizer, metrics=['mean_squared_error'])
  return model

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
dataset = pd.read_csv('/content/drive/MyDrive/DeepLearning/ANN/Complete_Deep_Learning/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1]
y = dataset.iloc[:, -1:]
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)
X = pd.concat([X, geography, gender], axis=1)
X = X.drop(['Geography', 'Gender'], axis=1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
model = KerasRegressor(model=create_model, batch_size=10, epochs=100, verbose=0, neurons=10, kernel_initializer='uniform')

In [26]:
neurons = [10, 20, 30, 40, 50, 60, 100]
#activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
kernel_initializer = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
loss = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error', 'cosine_similarity']
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#metrics = ['mean_squared_error', 'root_mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error']

param = dict(neurons=neurons, kernel_initializer=kernel_initializer, optimizer=optimizer, loss=loss)

random = RandomizedSearchCV(estimator=model, param_distributions=param, cv=3, n_jobs=-1)

In [27]:
random_result = random.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 726, in fit
    X=X, y=y, sample_weight=sample_weight, warm_start=self.warm_start, **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 888, in _fit
    self._check_model_compatibility(y)
  File "/usr/local/lib/p

In [28]:
print(random_result.best_params_, random_result.best_score_)

{'optimizer': 'Adam', 'neurons': 10, 'loss': 'mean_absolute_percentage_error', 'kernel_initializer': 'normal'} -0.2563089221127963


In [29]:
y_pred = random.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7975